In [2]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

2025/08/29 12:40:47 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/08/29 12:40:47 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


AttributeError: 'MlflowClient' object has no attribute 'list_experiments'

In [7]:
client.create_experiment(name="my-cool-experiment")

MlflowException: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-cool-experiment', None, 'active', 1756471345771, 1756471345771)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 78f3ed368b3c42319e04b1317b615621, rmse: 2.0752
run id: 152f010354d541359e8773aa784bbdc2, rmse: 2.4481
run id: 6c7a849a1f7847dfb648ef8d7bca64f0, rmse: 2.5895
run id: 85d1be8987b546b2b64ade6445a70c47, rmse: 2.6996
run id: b02a72e06b85416390df8dc053ef340e, rmse: 3.3196


In [10]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [11]:
run_id = "78f3ed368b3c42319e04b1317b615621"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

2025/08/29 12:43:24 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/08/29 12:43:24 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Successfully registered model 'nyc-taxi-regressor'.


MlflowException: Unable to find a logged_model with artifact_path model under run 78f3ed368b3c42319e04b1317b615621

In [12]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

/tmp/ipykernel_2777/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [13]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_2777/1718116654.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


MlflowException: Model Version (name=nyc-taxi-regressor, version=4) not found

In [14]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

MlflowException: Model Version (name=nyc-taxi-regressor, version=4) not found

In [15]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [17]:
df = read_dataframe("data/green_tripdata_2021-01.csv")

/tmp/ipykernel_2777/3050441246.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [18]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

MlflowException: Failed to download artifacts from path 'preprocessor', please ensure that the path is correct.

In [19]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessor/preprocessor.b'

In [20]:
X_test = preprocess(df, dv)

NameError: name 'dv' is not defined

In [21]:
target = "duration"
y_test = df[target].values

In [22]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

NameError: name 'X_test' is not defined

In [23]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

NameError: name 'X_test' is not defined

In [24]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_2777/3610659809.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


MlflowException: Model Version (name=nyc-taxi-regressor, version=4) not found